# Arrow, Pandas, Polars and PySpark - simple Comparison
In this notebook, we want to take a closer look at the concepts and learn which one is best suited for which scenario.
Credits:
- https://amanjaiswalofficial.medium.com/apache-arrow-making-spark-even-faster-3ae-8ca8e1a67dc7
- https://www.datacamp.com/de/tutorial/apache-arrow

In [1]:
# required imports
import time
from time import perf_counter
import pandas as pd
import numpy as np
import polars as pl
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
RUN_SPARK = True

## Generate Test Data
### lets first generate some test data
you can always adjust the size by adjusting `NUM_ROWS`

In [12]:
NUM_ROWS = 5000000 # kann angepasst werden

# %%
# Datensatz erzeugen
np.random.seed(42)
id = np.arange(1, NUM_ROWS+1)
x = np.random.randn(NUM_ROWS) * 10 + 50
y = np.random.randint(1, 5, size=NUM_ROWS)
target = 2*x + y + np.random.randn(NUM_ROWS)*5

df_pd = pd.DataFrame({
    'id': id,
    'x': x,
    'category': y,
    'target': target
})

df_pd.to_csv('large_dataset.csv', index=False)

In [9]:
# Timer
def timed(name, func):
    start = perf_counter()
    result = func()
    end = perf_counter()
    print(f"{name}: {end-start:.4f} s")
    return result

## Arrow

"Apache Arrow is a multi-language toolbox for building high performance applications that process and transport large data sets. It is designed to both improve the performance of analytical algorithms and the efficiency of moving data from one system or programming language to another.

A critical component of Apache Arrow is its in-memory columnar format, a standardized, language-agnostic specification for representing structured, table-like datasets in-memory. This data format has a rich data type system (included nested and user-defined data types) designed to support the needs of analytic database systems, data frame libraries, and more". [https://arrow.apache.org/overview/]

### Why Arrow
- Arrow uses its in-memory format, which has memory buffers organized in columns and batches. 
- this makes vectorised processing possible, performing operations on entire columns efficiently

### Shared Memory Model
- Arrow avoids traditional serialization
- Instead, it relies on a shared memory model in which multiple processes can directly access the same data without copying or converting it.

### `to_pandas` runtime comparison

In [30]:
spark = SparkSession.builder.appName("Runtime_Comparison").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
df = spark.read.csv("large_dataset.csv", header=True, inferSchema=True)

In [33]:
#pdf = timed("Without Arrow to pdf:", lambda:df.toPandas())
spark_df = timed("Without Arrow to spark df:", lambda:spark.createDataFrame(pdf))

Without Arrow to pdf:: 58.6861 s
Without Arrow to spark df:: 168.0553 s


In [34]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
#spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 1000)
#pdf = timed("With Arrow to pdf:", lambda:df.toPandas())
spark_df = timed("With Arrow to spark df:", lambda:spark.createDataFrame(pdf))

With Arrow to spark df:: 5.2246 s


In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark_df = spark.createDataFrame(pdf)

### Pandas Polars conversion runtime comparison

In [39]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
pl_df_arrow = timed("Without Arrow to polars df:", lambda:spark.createDataFrame(pdf))

Without Arrow to polars df:: 172.0041 s


In [40]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
pl_df_arrow = timed("Without Arrow to polars df:", lambda:spark.createDataFrame(pdf))

Without Arrow to polars df:: 5.7771 s


### PySpark Polars conversion runtime comparison

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
arrow_table = spark_df._collect_as_arrow()  # intern Arrow Table
pl_df_arrow = timed("Without Arrow to polars df:", lambda:spark_df._collect_as_arrow())
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
pl_df_arrow = timed("With Arrow to polars df:", lambda:spark_df._collect_as_arrow())

# `fillna` and `groupBy` Comparison

## Pandas: Runtime

In [35]:
#df = pd.DataFrame({'id': id, 'x': x, 'category': y, 'target': target})

print("--- pandas ---")
pdf = timed('pandas_fillna', lambda: pdf.fillna(0))
df_group = timed('pandas_groupby', lambda: pdf.groupby('category')['target'].mean())
print(df_group)

--- pandas ---
pandas_fillna: 0.2920 s
pandas_groupby: 0.2242 s
category
1    100.988890
2    101.991171
3    103.010719
4    103.985833
Name: target, dtype: float64


## Polars: Runtime

In [22]:
def polars_groupby():
    return df_pl.select([
        pl.col("category"),
        pl.col("target").mean().over("category").alias("mean_target")
    ]).unique(subset="category")

print("--- polars ---")
df_pl = df_pl = pl.from_pandas(df)
df_pd = timed('polars_fillna', lambda: df_pl.fill_null(0))
df_group_pl = timed('polars_groupby', polars_groupby)
print(df_group_pl)

--- polars ---
polars_fillna: 0.0401 s
polars_groupby: 3.7003 s
shape: (4, 2)
┌──────────┬─────────────┐
│ category ┆ mean_target │
│ ---      ┆ ---         │
│ i32      ┆ f64         │
╞══════════╪═════════════╡
│ 3        ┆ 103.001336  │
│ 4        ┆ 103.990736  │
│ 1        ┆ 101.00083   │
│ 2        ┆ 101.997864  │
└──────────┴─────────────┘


## PySpark: Runtime

In [19]:
df_pd = pd.DataFrame({
    'id': id,
    'x': x,
    'category': y,
    'target': target
})

print("--- pyspark ---")
spark = SparkSession.builder.master('local[*]').appName('SimpleCompare').getOrCreate()
sdf = timed('spark_from_pandas', lambda: spark.createDataFrame(df))
sdf = timed('spark_fillna', lambda: sdf.na.fill(0))
sdf_group = timed('spark_groupby', lambda: sdf.groupBy('category').agg(F.mean('target').alias('mean')))
sdf_group.show(5)
spark.stop()


--- pyspark ---
spark_from_pandas: 186.8315 s
spark_fillna: 0.0650 s
spark_groupby: 0.1378 s
+--------+------------------+
|category|              mean|
+--------+------------------+
|       1|100.98889018639353|
|       3|103.01071857029602|
|       2|101.99117119627803|
|       4|103.98583261511732|
+--------+------------------+

